In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, SimpleRNN, Dense, TimeDistributed, Dropout
from tensorflow.keras.models import Model

# Parameters for STFT/ISTFT and target audio length
FRAME_LENGTH = 1024
FRAME_STEP   = 256
FFT_LENGTH   = 1024
TARGET_LENGTH = 16000  # desired output length
NUM_FREQ_BINS = FFT_LENGTH // 2 + 1  # should be 513

# STFT function without the 'center' argument.
def stft_layer(x):
    return tf.signal.stft(x,
                          frame_length=FRAME_LENGTH,
                          frame_step=FRAME_STEP,
                          fft_length=FFT_LENGTH,
                          window_fn=tf.signal.hann_window,
                          pad_end=False)

# ISTFT function without the 'center' argument.
def istft_layer(x):
    return tf.signal.inverse_stft(x,
                                  frame_length=FRAME_LENGTH,
                                  frame_step=FRAME_STEP,
                                  fft_length=FFT_LENGTH,
                                  window_fn=tf.signal.hann_window)

# Custom layer to pad or crop the output to the TARGET_LENGTH.
class PadToTarget(tf.keras.layers.Layer):
    def __init__(self, target_length, **kwargs):
        self.target_length = target_length
        super(PadToTarget, self).__init__(**kwargs)

    def call(self, x):
        current_length = tf.shape(x)[1]
        pad_len = self.target_length - current_length
        return tf.cond(pad_len > 0,
                       lambda: tf.pad(x, [[0, 0], [0, pad_len]]),
                       lambda: x[:, :self.target_length])

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.target_length)

# Input: raw audio waveform of fixed length TARGET_LENGTH.
audio_input = Input(shape=(TARGET_LENGTH,), name="audio_input")

# 1. Compute STFT.
complex_spec = Lambda(stft_layer,
                      output_shape=lambda s: (s[0], None, NUM_FREQ_BINS),
                      name="stft")(audio_input)

# 2. Split into magnitude and phase components.
mag = Lambda(lambda x: tf.abs(x), name="magnitude")(complex_spec)
phase = Lambda(lambda x: tf.math.angle(x), name="phase")(complex_spec)

# 3. Process the magnitude spectrogram using two SimpleRNN layers.
rnn = SimpleRNN(128, return_sequences=True, name="rnn_1")(mag)
rnn = Dropout(0.2, name="dropout_1")(rnn)
rnn = SimpleRNN(128, return_sequences=True, name="rnn_2")(rnn)
rnn = Dropout(0.2, name="dropout_2")(rnn)

# Use a fixed number of frequency bins (NUM_FREQ_BINS) for the Dense layer.
mask = TimeDistributed(Dense(NUM_FREQ_BINS, activation='relu'),
                       name="mask")(rnn)

# Apply the predicted mask to the original magnitude.
restored_mag = Lambda(lambda x: x[0] * x[1],
                      output_shape=lambda s: s[0],
                      name="apply_mask")([mag, mask])

# 4. Rebuild the complex spectrogram using the original phase.
restored_complex = Lambda(lambda x: tf.complex(x[0] * tf.math.cos(x[1]),
                                               x[0] * tf.math.sin(x[1])),
                            output_shape=lambda s: s[0],
                            name="rebuild_complex")([restored_mag, phase])

# 5. Inverse STFT to convert back to time-domain.
restored_audio = Lambda(istft_layer,
                        output_shape=lambda s: (s[0], None),
                        name="istft")(restored_complex)

# 6. Pad or crop the output so that it exactly matches TARGET_LENGTH.
final_output = PadToTarget(TARGET_LENGTH, name="pad_output")(restored_audio)

# Build and compile the model.
model = Model(inputs=audio_input, outputs=final_output, name="ClippingRestorationModel")
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


Model: "ClippingRestorationModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ audio_input (InputLayer)  │ (None, 16000)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stft (Lambda)             │ (None, None, 513)      │              0 │ audio_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ magnitude (Lambda)        │ (None, None, 513)      │              0 │ stft[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rnn_1 (SimpleRNN)         │ (None, None, 128)      │         82,176 │ magnitude[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, None, 128)      │              0 │ rnn_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rnn_2 (SimpleRNN)         │ (None, None, 128)      │         32,896 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, None, 128)      │              0 │ rnn_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ mask (TimeDistributed)    │ (None, None, 513)      │         66,177 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ apply_mask (Lambda)       │ (None, None, 513)      │              0 │ magnitude[0][0],       │
│                           │                        │                │ mask[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ phase (Lambda)            │ (None, None, 513)      │              0 │ stft[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rebuild_complex (Lambda)  │ (None, None, 513)      │              0 │ apply_mask[0][0],      │
│                           │                        │                │ phase[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ istft (Lambda)            │ (None, None)           │              0 │ rebuild_complex[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pad_output (PadToTarget)  │ (None, 16000)          │              0 │ istft[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 181,249 (708.00 KB)

 Trainable params: 181,249 (708.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import wave
import numpy as np

def read_wav_as_float(file_path):
    """
    Reads a WAV file and returns its samples as a list of floating-point values.

    Parameters:
        file_path (str): Path to the WAV file.

    Returns:
        list: A list of floating-point samples.
    """
    # Open the WAV file
    with wave.open(file_path, 'rb') as wav_file:
        # Get parameters
        n_channels = wav_file.getnchannels()
        sample_width = wav_file.getsampwidth()
        n_frames = wav_file.getnframes()
        frame_rate = wav_file.getframerate()

        print(f"Channels: {n_channels}, Sample Width: {sample_width}, Frame Rate: {frame_rate}, Frames: {n_frames}")

        # Read frames as bytes
        raw_data = wav_file.readframes(n_frames)

    # Determine the data type based on sample width
    dtype = {1: np.int8, 2: np.int16, 4: np.int32}.get(sample_width)
    if dtype is None:
        raise ValueError(f"Unsupported sample width: {sample_width}")

    # Convert raw bytes to numpy array
    int_data = np.frombuffer(raw_data, dtype=dtype)

    # Normalize to floating-point range [-1.0, 1.0]
    max_val = float(2 ** (8 * sample_width - 1))
    float_data = int_data / max_val

    # Handle multi-channel audio by averaging channels
    if n_channels > 1:
        float_data = float_data.reshape(-1, n_channels).mean(axis=1)

    return float_data.tolist()

# Example usage
wav_file_path = 'words.wav'  # Replace with the path to your WAV file
wav_file_path1 = 'wordsc.wav'
samples = read_wav_as_float(wav_file_path)
samples1 = read_wav_as_float(wav_file_path1)

Channels: 1, Sample Width: 2, Frame Rate: 44100, Frames: 711007
Channels: 1, Sample Width: 2, Frame Rate: 44100, Frames: 711007


In [ ]:
import numpy as np

X_train = np.array(samples[0:(len(samples)//16000)*16000]).reshape((len(samples)//16000,16000)).astype(np.float32)
Y_train = np.array(samples1[0:(len(samples1)//16000)*16000]).reshape((len(samples)//16000,16000)).astype(np.float32)

model.fit(X_train, Y_train, epochs=150, batch_size=16)

Epoch 1/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 65s 127ms/step - loss: 0.0104
Epoch 2/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - loss: 0.0074
Epoch 3/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step - loss: 0.0049
Epoch 4/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.0044
Epoch 5/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 0.0038
Epoch 6/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - loss: 0.0038
Epoch 7/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.0032
Epoch 8/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.0033
Epoch 9/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - loss: 0.0031
Epoch 10/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.0032
Epoch 11/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - loss: 0.0030
Epoch 12/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - loss: 0.0031
Epoch 13/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - loss: 0.0028
Epoch 14/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - loss: 0.0028
Epoch 15/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - loss: 0.0025
Epo

In [ ]:
samples_clipped = read_wav_as_float("1c.wav")
import numpy as np

# Suppose you have a single raw audio sample in a NumPy array:
raw_audio = np.array(samples_clipped[0:-(len(samples_clipped) % 16000)]).reshape((len(samples_clipped)//16000,16000))

# Add a batch dimension: shape becomes (1, audio_length)
#raw_audio_batch = np.expand_dims(raw_audio, axis=0)
restored_audio = np.empty(raw_audio.shape)
for i in range(len(samples_clipped)//16000):
  # Use the model to predict (restore) the audio
  restored_audio_batch = model.predict(raw_audio[i].reshape((1,16000)))

  # Remove the batch dimension to get the restored audio waveform
  restored_audio[i] = restored_audio_batch[0]

print("Restored audio shape:", restored_audio.shape)
restored_audio = restored_audio.flatten()

Channels: 1, Sample Width: 2, Frame Rate: 44100, Frames: 122880
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Restored audio shape: (7, 16000)


In [ ]:
def write_float_samples_to_wav(samples, sample_rate, output_path):
    """
    Writes floating-point audio samples to a mono 16-bit WAV file.

    Parameters:
        samples (list or np.ndarray): Array of floating-point audio samples in the range [-1.0, 1.0].
        sample_rate (int): Sample rate of the audio in Hz (e.g., 44100).
        output_path (str): Path to save the output WAV file.
    """
    # Ensure the samples are a NumPy array
    samples = np.array(samples, dtype=np.float32)

    # Clip the samples to the range [-1.0, 1.0] to prevent overflow
    samples = np.clip(samples, -1.0, 1.0)

    # Convert to 16-bit PCM format
    int_samples = (samples * 32767).astype(np.int16)

    # Write to a WAV file
    with wave.open(output_path, 'wb') as wav_file:
        # Set the parameters for the WAV file
        wav_file.setnchannels(1)  # Mono
        wav_file.setsampwidth(2)  # 16-bit PCM
        wav_file.setframerate(sample_rate)

        # Write the audio frames
        wav_file.writeframes(int_samples.tobytes())

In [ ]:
write_float_samples_to_wav(restored_audio,44100,'output.wav')